In [1]:
import torch
from torch import nn
from torchinfo import summary

from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import lightning as L
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.optim as optim

from tqdm.auto import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy  as np
import tracemalloc 

import json
import os
import glob
import pickle
from itertools import combinations
import gc
import time
import random
import sys

sys.path.insert(0, os.path.join(os.path.abspath(''), ".."))
from model.builder import Classifier1DMaxPoolBNModel
from datamodule.datamodule import DataModule, FFTDataModule_1_sensor

import warnings
warnings.filterwarnings("ignore")

/home/tran/miniconda3/envs/compgan/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
L.seed_everything(42)

Seed set to 42


42

In [3]:
TEST = False

In [4]:
n_epochs = 20000000
patience = n_epochs//10000
missing = 6
user = 0

batch_size = 512
log_save_dir = os.path.join("../data/compgan_dataset/", "results", "84_classify_1_sensor")
log_save_name = f"{missing}_missing/user{user}"

In [5]:
data_module = FFTDataModule_1_sensor(
    test_user=user, 
    missing_sensor_numbers=missing,
    batch_size=512,
    test_mode=True)

In [6]:
from lightning.pytorch.utilities.model_summary import ModelSummary

net = Classifier1DMaxPoolBNModel(
    cnn_channel_param = [
        (42, 64, 8, 0, 3),
        (64, 128, 8, 0, 3)
    ],
    pool_channel_param = [
        None, None,
    ],
    linear_channel_param = [
        1024, 256, 128
    ],
    out_class_num=10,
    in_feature_shape=129)

model_summary = ModelSummary(net, max_depth=6)
print(model_summary)

   | Name      | Type        | Params | In sizes      | Out sizes    
---------------------------------------------------------------------------
0  | cnn       | Sequential  | 87.6 K | [10, 42, 256] | [10, 128, 26]
1  | cnn.0     | Conv1d      | 21.6 K | [10, 42, 256] | [10, 64, 83] 
2  | cnn.1     | BatchNorm1d | 128    | [10, 64, 83]  | [10, 64, 83] 
3  | cnn.2     | ReLU        | 0      | [10, 64, 83]  | [10, 64, 83] 
4  | cnn.3     | Conv1d      | 65.7 K | [10, 64, 83]  | [10, 128, 26]
5  | cnn.4     | BatchNorm1d | 256    | [10, 128, 26] | [10, 128, 26]
6  | cnn.5     | ReLU        | 0      | [10, 128, 26] | [10, 128, 26]
7  | linear    | Sequential  | 9.2 M  | [10, 3328]    | [10, 10]     
8  | linear.0  | Linear      | 6.8 M  | [10, 3328]    | [10, 2048]   
9  | linear.1  | BatchNorm1d | 4.1 K  | [10, 2048]    | [10, 2048]   
10 | linear.2  | ReLU        | 0      | [10, 2048]    | [10, 2048]   
11 | linear.3  | Linear      | 2.1 M  | [10, 2048]    | [10, 1024]   
12 | linear.4 

In [8]:
print(" ----------------------start training---------------------------")
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

tensorboard_logger = TensorBoardLogger(save_dir=log_save_dir, name=log_save_name,)
checkpoint_callback = ModelCheckpoint(
    dirpath=None,
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    filename="sample_{epoch:02d}-{step:02d}-{val_loss:02f}"
)

trainer = L.Trainer(
    logger=[tensorboard_logger],
    callbacks=[EarlyStopping(monitor="val_loss", patience=patience), checkpoint_callback],
    max_epochs=n_epochs,
    check_val_every_n_epoch=10,
    accelerator="gpu", 
    )

trainer.fit(model=net, datamodule=data_module)
trainer_test_dict = trainer.logged_metrics

trainer.test(model=net, datamodule=data_module)
trainer_test_dict.update(trainer.logged_metrics)

print("trainer.logger.log_dir", trainer.logger.log_dir)

for key in trainer_test_dict.keys():
    trainer_test_dict[key] = trainer_test_dict[key].item()

with open(os.path.join(trainer.logger.log_dir, "result.json"), "w") as f:
    json.dump(trainer_test_dict, f, indent=4)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


 ----------------------start training---------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params | In sizes      | Out sizes    
----------------------------------------------------------------------
0 | cnn    | Sequential | 87.6 K | [10, 42, 256] | [10, 128, 26]
1 | linear | Sequential | 9.2 M  | [10, 3328]    | [10, 10]     
----------------------------------------------------------------------
9.3 M     Trainable params
0         Non-trainable params
9.3 M     Total params
37.228    Total estimated model params size (MB)


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  4.70it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  4.68it/s]
teardown


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 184.62it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.06542056053876877
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
teardown
trainer.logger.log_dir None


TypeError: expected str, bytes or os.PathLike object, not NoneType